In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf
spark = SparkSession.builder.\
            master("local").\
            appName("amzone").\
            enableHiveSupport().\
            getOrCreate()
sc = spark.sparkContext
# inputPath="data/sample.json"
# rawdata.select("reviewerID","asin","reviewerName","overall","reviewTime").show()
inputPath = "data/reviews_Books_5.json"
rawdata = spark.read.json(inputPath)
ratingDf = rawdata.select(rawdata.reviewerID.alias("user"),\
               rawdata.asin.alias("item"),\
               rawdata.overall.alias("rating"))
ratingDf.registerTempTable("Ratings")
sqlString="with t1 as (select count(distinct(user)) as users from Ratings),\
           t2 as (select count(distinct(item)) as items from Ratings), \
           t3 as (select count(distinct(rating)) as ratings from Ratings) \
            select * from t1 cross join t2 cross join t3"
# rawdata.select("reviewerID").distinct().count()
resultDf = spark.sql(sqlString)
resultDf.show()
trainingData,cvData,testData = ratingDf.randomSplit([0.6,0.2,0.2])
# resultDf.show()
from pyspark.ml.pipeline import Pipeline,PipelineModel
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import *
userIndexer = StringIndexer(inputCol="user",outputCol="userid",handleInvalid="error").fit(ratingDf)
itemIndexer = StringIndexer(inputCol="item",outputCol="itemid").fit(ratingDf)
alsModel = ALS(rank=5,userCol="userid",itemCol="itemid",ratingCol="rating",maxIter=5,regParam=0.0001)
pipeAls = Pipeline(stages=[userIndexer,itemIndexer,alsModel])
pipeAlsModel = pipeAls.fit(trainingData)
predictionAls = pipeAlsModel.transform(trainingData)
predictionAls.sort(asc("userid")).limit(10).show()
prediction = pipeAlsModel.transform(testData)
prediction.sort(asc("userid")).limit(10).show()
evaluator = RegressionEvaluator()\
      .setMetricName("rmse")\
      .setLabelCol("rating")\
      .setPredictionCol("prediction")
rmse = evaluator.evaluate(prediction.na.drop())
print "RMSE:",rmse
sc.stop()
spark.stop()

In [25]:
# resultDf.show()
from pyspark.ml.pipeline import Pipeline,PipelineModel
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import *
from pyspark.sql.functions import *
inputPath="data/sample.json"
# rawdata.select("reviewerID","asin","reviewerName","overall","reviewTime").show()
# inputPath = "data/reviews_Books_5.json"
rawdata = spark.read.json(inputPath)
ratingDf = rawdata.select(rawdata.reviewerID.alias("user"),\
               rawdata.asin.alias("item"),\
               rawdata.overall.alias("rating"))
ratingDf = ratingDf.select(ratingDf.user,ratingDf.item,ratingDf.rating.cast(FloatType()))
ratingDf.registerTempTable("Ratings")
sqlString="with t1 as (select count(distinct(user)) as users from Ratings),\
           t2 as (select count(distinct(item)) as items from Ratings), \
           t3 as (select count(distinct(rating)) as ratings from Ratings) \
            select * from t1 cross join t2 cross join t3"
# rawdata.select("reviewerID").distinct().count()
resultDf = spark.sql(sqlString)
trainingData,cvData,testData = ratingDf.randomSplit([0.6,0.2,0.2])
userIndexer = StringIndexer(inputCol="user",outputCol="userid",handleInvalid="error").fit(ratingDf)
itemIndexer = StringIndexer(inputCol="item",outputCol="itemid").fit(ratingDf)
alsModel = ALS(rank=5,userCol="userid",itemCol="itemid",ratingCol="rating",maxIter=5,regParam=0.0001)
pipeAls = Pipeline(stages=[userIndexer,itemIndexer,alsModel])
pipeAlsModel = pipeAls.fit(trainingData)
predictionAls = pipeAlsModel.transform(trainingData)
predictionRecommendation= pipeAlsModel.transform(testData)
evaluator = RegressionEvaluator()\
      .setMetricName("rmse")\
      .setLabelCol("rating")\
      .setPredictionCol("prediction")
resultDf.show()
predictionAls.sort(asc("userid")).limit(10).show()
predictionRecommendation.sort(asc("userid")).limit(10).show()
rmse = evaluator.evaluate(predictionRecommendation.na.drop())
print "RMSE:",rmse

+-----+-----+-------+
|users|items|ratings|
+-----+-----+-------+
|  100|    1|      5|
+-----+-----+-------+

+--------------------+----------+------+------+------+----------+
|                user|      item|rating|userid|itemid|prediction|
+--------------------+----------+------+------+------+----------+
|       A1FQPOYRBTTK1|000100039X|   5.0|   0.0|   0.0|  4.999894|
|      A3FFNE1DR5SI1W|000100039X|   5.0|   1.0|   0.0|  4.999894|
|A10000012B7CGYKOM...|000100039X|   5.0|   2.0|   0.0|  4.999894|
|      A1NPNGWBVD9AK3|000100039X|   5.0|   3.0|   0.0|  4.999894|
|        A9WX8DK93SN5|000100039X|   1.0|   4.0|   0.0| 0.9999787|
|      A1MOSTXNIO5MPJ|000100039X|   5.0|   5.0|   0.0|  4.999894|
|      A2BQZRA2P81BQG|000100039X|   2.0|   6.0|   0.0| 1.9999574|
|      A3SK701OIJM1YX|000100039X|   5.0|   7.0|   0.0|  4.999894|
|      A2X6GEC6LCDN4S|000100039X|   5.0|  11.0|   0.0|  4.999894|
|      A3FI0744PG1WYG|000100039X|   5.0|  14.0|   0.0|  4.999894|
+--------------------+---------

Py4JJavaError: An error occurred while calling o989.evaluate.
: org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.ScalaUDF.eval(ScalaUDF.scala:1072)
	at org.apache.spark.sql.catalyst.expressions.EqualNullSafe.eval(predicates.scala:470)
	at org.apache.spark.sql.catalyst.optimizer.EliminateOuterJoin$.org$apache$spark$sql$catalyst$optimizer$EliminateOuterJoin$$canFilterOutNull(joins.scala:116)
	at org.apache.spark.sql.catalyst.optimizer.EliminateOuterJoin$$anonfun$7.apply(joins.scala:125)
	at org.apache.spark.sql.catalyst.optimizer.EliminateOuterJoin$$anonfun$7.apply(joins.scala:125)
	at scala.collection.LinearSeqOptimized$class.exists(LinearSeqOptimized.scala:93)
	at scala.collection.immutable.List.exists(List.scala:84)
	at org.apache.spark.sql.catalyst.optimizer.EliminateOuterJoin$.org$apache$spark$sql$catalyst$optimizer$EliminateOuterJoin$$buildNewJoinType(joins.scala:125)
	at org.apach

In [ ]:
ratingDf = ratingDf.select(ratingDf.user,ratingDf.item,ratingDf.rating.cast(FloatType())

In [30]:
predictionRecommendation.na.fill(9).show()

+--------------+----------+------+------+------+----------+
|          user|      item|rating|userid|itemid|prediction|
+--------------+----------+------+------+------+----------+
| ABFOAYZA2UHD3|000100039X|   5.0|  65.0|   0.0|       9.0|
|A3A0ZK5GTHSCUV|000100039X|   5.0|  76.0|   0.0|       9.0|
| AFY0BT42DDYZV|000100039X|   5.0|  91.0|   0.0|       9.0|
|A1XLZQ205ZAC7L|000100039X|   5.0|  22.0|   0.0|       9.0|
|A2OP1HD9RGX5OW|000100039X|   5.0|  52.0|   0.0|       9.0|
|A3H65DAAV98C8F|000100039X|   5.0|  13.0|   0.0|       9.0|
|A2E71VWXO59342|000100039X|   5.0|  86.0|   0.0|       9.0|
| A1DQYPYI53ABM|000100039X|   5.0|  57.0|   0.0|       9.0|
|A2ZB1G1KUE6OS6|000100039X|   2.0|  19.0|   0.0|       9.0|
|A1MCAHDE1F3Q6L|000100039X|   5.0|  64.0|   0.0|       9.0|
| AAFLZI7MX9UIG|000100039X|   5.0|  15.0|   0.0|       9.0|
| A5E9TSD20U9PR|000100039X|   5.0|  61.0|   0.0|       9.0|
| AMRZ5G7HF7I03|000100039X|   5.0|   9.0|   0.0|       9.0|
|A2I35JB67U20C0|000100039X|   5.0|  17.0

In [10]:
ratingDf.show()

+--------------------+----------+------+
|              userid|    itemid|rating|
+--------------------+----------+------+
|A10000012B7CGYKOM...|000100039X|   5.0|
|      A2S166WSCFIFP5|000100039X|   5.0|
|      A1BM81XB4QHOA3|000100039X|   5.0|
|      A1MOSTXNIO5MPJ|000100039X|   5.0|
|      A2XQ5LZHTD4AFT|000100039X|   5.0|
|      A3V1MKC2BVWY48|000100039X|   5.0|
|      A12387207U8U24|000100039X|   5.0|
|      A29TRDMK51GKZR|000100039X|   5.0|
|      A3FI0744PG1WYG|000100039X|   5.0|
|      A2LBBQHYLEHM7P|000100039X|   5.0|
|      A1340OFLZBW5NG|000100039X|   5.0|
|      A2KU9IU07LOJS1|000100039X|   5.0|
|      A2WVHIRDMLM82E|000100039X|   5.0|
|      A2I35JB67U20C0|000100039X|   5.0|
|      A19N3FCQCLJYUA|000100039X|   5.0|
|      A3FFNE1DR5SI1W|000100039X|   5.0|
|      A1TT4CY55WLHAR|000100039X|   5.0|
|      A2X4HE21JTAL98|000100039X|   5.0|
|       ARDQ9KNB8K22N|000100039X|   5.0|
|       A27ZH1AQORJ1L|000100039X|   5.0|
+--------------------+----------+------+
only showing top

In [52]:
inputPath="data/ratings_Electronics_sample.csv"
rawdata = spark.read.csv(inputPath)
df = rawdata.select(rawdata._c0.alias("user"),\
                       rawdata._c1.alias("item"),\
                       rawdata._c2.astype("float").alias("rating"),\
                       rawdata._c3.alias("timestamp"))
trainingData,cvData,testData = df.randomSplit([0.6,0.2,0.2])

In [54]:
from pyspark.ml.pipeline import Pipeline,PipelineModel
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import *
userIndexer = StringIndexer(inputCol="user",outputCol="userid",handleInvalid="error").fit(df)
itemIndexer = StringIndexer(inputCol="item",outputCol="itemid").fit(df)
alsModel = ALS(rank=5,userCol="userid",itemCol="itemid",ratingCol="rating",maxIter=5,regParam=0.0001)
pipeAls = Pipeline(stages=[userIndexer,itemIndexer,alsModel])
pipeAlsModel = pipeAls.fit(trainingData)

predictionAls = pipeAlsModel.transform(trainingData)
predictionAls.sort(asc("userid")).show()

prediction = pipeAlsModel.transform(testData)
prediction.sort(asc("userid")).show()


+--------------+----------+------+----------+------+------+----------+
|          user|      item|rating| timestamp|userid|itemid|prediction|
+--------------+----------+------+----------+------+------+----------+
|A2SRW2W1GXYKQZ|0594481813|   5.0|1403827200|   1.0|   1.0|  4.999883|
|A2OC1UK7VMBLOT|0594481813|   4.0|1398643200|   2.0|   1.0| 3.9999063|
| A6DI4IE8MKFYR|0972683275|   5.0|1363219200|   3.0|   0.0|  4.999068|
|A106YUCY4SVX1D|0972683275|   5.0|1310083200|   4.0|   0.0|  4.999068|
|A3OYE7X2O08LNT|0972683275|   4.0|1360195200|   5.0|   0.0| 3.9992542|
|A2ULCET06LOPBB|0972683275|   5.0|1369785600|   6.0|   0.0|  4.999068|
|A3M122DYN9L5N8|0594451647|   1.0|1388707200|   8.0|   4.0| 0.9999989|
|A3IIGCFLKVFW8M|0972683275|   5.0|1393459200|   9.0|   0.0|  4.999068|
|A39Z4OU2C7ENWH|0972683275|   3.0|1328572800|  10.0|   0.0| 2.9994407|
|A3T3XKC7H5ACI1|0594033934|   5.0|1401235200|  12.0|  16.0|   4.99998|
|A1RPEK98P97J7W|0972683275|   5.0|1359504000|  14.0|   0.0|  4.999068|
|A3S0R

In [49]:
from pyspark.sql.functions import *
prediction.select("prediction").distinct().count()

1